In [ ]:
import geopandas as gpd, folium
from folium import plugins
import osmnx as ox

In [ ]:
deaths_gdf = gpd.read_file('data/deaths_gdf.gpkg')

In [ ]:
deaths_gdf

In [ ]:
deaths_coords = (deaths_gdf.centroid.y.mean(), deaths_gdf.centroid.x.mean())

In [ ]:
footprints_centroid_gdf = gpd.read_file('data/footprints_centroids_gdf.gpkg')

footprints_centroid_gdf

In [ ]:
buffer = 0.001

deaths_polygon = deaths_gdf.geometry.to_crs('epsg:4326').buffer(buffer).unary_union

deaths_polygon

In [ ]:
polygon_gdf = gpd.GeoDataFrame(geometry=[deaths_polygon], crs='epsg:4326')

polygon_gdf

In [ ]:
%%time
points_within_gdf = gpd.sjoin(footprints_centroid_gdf, polygon_gdf, op='within').drop(columns=['index_right'])

In [ ]:
points_within_gdf['centroid_x'] = points_within_gdf.geometry.to_crs('epsg:4326').centroid.x
points_within_gdf['centroid_y'] = points_within_gdf.geometry.to_crs('epsg:4326').centroid.y

In [ ]:
points_within_gdf

In [ ]:
points_within_proj_gdf = points_within_gdf.to_crs('epsg:32630').copy()
deaths_proj_gdf = deaths_gdf.to_crs('epsg:32630').copy()

In [ ]:
points_within_proj_gdf

In [ ]:
points_within_proj_gdf['x'] = points_within_proj_gdf.geometry.x
points_within_proj_gdf['y'] = points_within_proj_gdf.geometry.y

points_within_proj_gdf

In [ ]:
deaths_proj_gdf['x'] = deaths_proj_gdf.geometry.x
deaths_proj_gdf['y'] = deaths_proj_gdf.geometry.y

deaths_proj_gdf

In [ ]:
%%time
from shapely.geometry import Point, LineString
import pandas as pd

dict_list = []
for drow in deaths_proj_gdf.itertuples():
    sorted_list = []
    s_dict = {}
    for frow in points_within_proj_gdf.itertuples():
        distance = ox.distance.euclidean_dist_vec(frow.y, frow.x, drow.y, drow.x)
        sorted_list.append([drow.Index, frow.Index, distance, drow.x, drow.y, frow.x, frow.y, frow.osmid, drow.DEATHS])
    s = sorted(sorted_list, key = lambda x: x[2])[0]
    orig_coord = Point(s[3], s[4])
    dest_coord = Point(s[5], s[6])
    line = LineString([orig_coord, dest_coord]).wkt
    s_dict = {'death_index': s[0],'footprint_index': s[1], 'distance_m': s[2], \
              'wkt_line': line, 'osmid': s[7],'deaths': s[8]}
    dict_list.append(s_dict)
df = pd.DataFrame(dict_list)

In [ ]:
df

In [ ]:
%%time
from shapely import wkt

df['geometry'] = df.wkt_line.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='epsg:32630')

In [ ]:
gdf

In [ ]:
linestrings_gdf = gdf.to_crs('epsg:4326').drop(columns=['wkt_line']).copy()

linestrings_gdf

In [ ]:
%%time
lasts = []
for line in linestrings_gdf.geometry:
    endpoints = line.boundary
    first, last = endpoints.geoms
    lasts.append(last.wkt)

In [ ]:
linestrings_gdf['wkt_dest_points'] = lasts

In [ ]:
linestrings_gdf

In [ ]:
linestrings_gdf.to_file('data/linestrings_gdf.gpkg', driver='GPKG')

In [ ]:
%%time
map1 = folium.Map(location=deaths_coords, tiles='CartoDB positron', zoom_start=17)

fg1=folium.FeatureGroup(name='Cholera deaths', show=True)
deaths_style = {'fillColor':'black','color':'black','weight':2,'opacity':0.5, 'fillOpacity':0.5}
deaths_marker = {'radius':5}
for row in deaths_gdf.itertuples():
    folium.CircleMarker(
                    location=[row.LAT,row.LON], \
                    radius=deaths_marker['radius'], \
                    color=deaths_style['color'], \
                    weight=deaths_style['weight'], \
                    fill=True, \
                    fill_color=deaths_style['fillColor'], \
                    fill_opacity=deaths_style['fillOpacity']).add_to(fg1)

fg1.add_to(map1)

fg2=folium.FeatureGroup(name='Building footprints', show=True)
footprints_style = {'fillColor':'red','color':'black','weight':1,'opacity':0.5, 'fillOpacity':0.5}
footprints_marker = {'radius':4}
for row in points_within_gdf.itertuples():
    folium.CircleMarker(
                    location=[row.centroid_y,row.centroid_x], \
                    radius=footprints_marker['radius'], \
                    color=footprints_style['color'], \
                    weight=footprints_style['weight'], \
                    fill=True, \
                    fill_color=footprints_style['fillColor'], \
                    fill_opacity=footprints_style['fillOpacity']).add_to(fg2)
    
fg2.add_to(map1)

fg3=folium.FeatureGroup(name='Nearest building', show=True)
folium.Choropleth(linestrings_gdf, line_weight=2, color='blue').add_to(fg3)
fg3.add_to(map1)

folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(map1)

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map1)

map1

In [ ]:
buildings_osmid_df = linestrings_gdf[['osmid','deaths','wkt_dest_points']].groupby(['osmid', 'wkt_dest_points'])['deaths'].sum().reset_index()

buildings_osmid_df

In [ ]:
linestrings_gdf[linestrings_gdf['osmid']==54666919]

In [ ]:
from shapely import wkt

buildings_osmid_df['geometry'] = buildings_osmid_df.wkt_dest_points.apply(wkt.loads)
buildings_osmid_gdf = gpd.GeoDataFrame(buildings_osmid_df, geometry='geometry', crs='epsg:4326').drop(columns=['wkt_dest_points'])

buildings_osmid_gdf

In [ ]:
buildings_osmid_gdf.to_file('data/buildings_osmid_gdf.gpkg', driver='GPKG')

In [ ]:
%%time
map1 = folium.Map(location=deaths_coords, tiles='CartoDB positron', zoom_start=17)

fg1=folium.FeatureGroup(name='Cholera deaths', show=True)
deaths_style = {'fillColor':'black','color':'black','weight':2,'opacity':0.5, 'fillOpacity':0.5}
deaths_marker = {'radius':6}
for row in buildings_osmid_gdf.itertuples():
    folium.CircleMarker(
                    location=(row.geometry.y, row.geometry.x), \
                    radius=deaths_marker['radius'], \
                    color=deaths_style['color'], \
                    weight=deaths_style['weight'], \
                    fill=True, \
                    fill_color=deaths_style['fillColor'], \
                    fill_opacity=deaths_style['fillOpacity']).add_to(fg1)

fg1.add_to(map1)

fg2=folium.FeatureGroup(name='Building footprints', show=True)
footprints_style = {'fillColor':'red','color':'black','weight':1,'opacity':0.5, 'fillOpacity':0.5}
footprints_marker = {'radius':4}
for row in points_within_gdf.itertuples():
    folium.CircleMarker(
                    location=[row.centroid_y,row.centroid_x], \
                    radius=footprints_marker['radius'], \
                    color=footprints_style['color'], \
                    weight=footprints_style['weight'], \
                    fill=True, \
                    fill_color=footprints_style['fillColor'], \
                    fill_opacity=footprints_style['fillOpacity']).add_to(fg2)
    
fg2.add_to(map1)

folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(map1)

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map1)

map1